In [1]:
# import all relevant libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns
sns.set()
sns.set_style("white")
sns.set_palette("GnBu_d")

# also import these "new" libraries 
# Note: you may have to download an add them to your environment (using e.g. 'conda install -c conda-forge folium')

import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime #for working with times objects
from datetime import timedelta #for working with times objects
import math
import random
import timeit

In [2]:
dsphil18 = pd.read_csv('philadelphia_2018.csv', low_memory=False)
dsStation1 = pd.read_csv('indego-trips-2018-q1.csv', low_memory=False)
dsStation2 = pd.read_csv('indego-trips-2018-q2.csv', low_memory=False)
dsStation3 = pd.read_csv('indego-trips-2018-q3.csv', low_memory=False)
dsStation4 = pd.read_csv('indego-trips-2018-q4.csv', low_memory=False)
dsStationsInfo = pd.read_csv('indego-stations-2021-01-01.csv', low_memory=False)

In [3]:
dsphil18.tail()

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name
671813,2018-12-31 23:18:00,2019-01-01 00:04:00,3037,3037,14519,Day Pass,Pennsylvania Convention Center,Pennsylvania Convention Center
671814,2018-12-31 23:25:00,2019-01-01 00:06:00,3168,3168,11891,Indego30,20th & Sansom,20th & Sansom
671815,2018-12-31 23:26:00,2019-01-01 06:44:00,3026,3026,5410,Indego30,2nd & South,2nd & South
671816,2018-12-31 23:33:00,2018-12-31 23:47:00,3063,3150,2701,Day Pass,17th & Pine,"15th & Castle, DiSilvestro Playground"
671817,2018-12-31 23:39:00,2018-12-31 23:49:00,3007,3049,3471,Indego30,"11th & Pine, Kahn Park",Foglietta Plaza


In [4]:
dsStation1.head()


,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type
0,223869188,18,2018-01-01 00:24:00,2018-01-01 00:42:00,3124,39.952950,-75.139793,3073,39.961430,-75.152420,3708,30,One Way,Indego30
1,223905597,572,2018-01-01 00:38:00,2018-01-01 10:10:00,3023,39.950481,-75.172859,3066,39.945610,-75.173477,3288,365,One Way,Indego365
2,223872811,22,2018-01-01 00:48:00,2018-01-01 01:10:00,3026,39.941380,-75.145638,3023,39.950481,-75.172859,11735,30,One Way,Indego30
3,223872810,21,2018-01-01 01:03:00,2018-01-01 01:24:00,3045,39.947922,-75.162369,3037,39.954239,-75.161377,5202,30,One Way,Indego30
4,223872809,4,2018-01-01 01:05:00,2018-01-01 01:09:00,3115,39.972630,-75.167572,3058,39.967159,-75.170013,5142,30,One Way,Indego30


In [5]:
#dsStation2.head()


In [6]:
#dsStation3.head()


In [7]:
#dsStation4.head()


In [23]:
dsStationsInfo['start_station']= dsStationsInfo['Station_ID']
dsStationsInfo['start_station_name']= dsStationsInfo['Station_Name']
dsStationsInfo['end_station']= dsStationsInfo['Station_ID']
dsStationsInfo['end_station_name']= dsStationsInfo['Station_Name']
dsStationsInfo.head()


,Station_ID,Station_Name,Go_live_date,Status,start_station,start_station_name,end_station,end_station_name,start_station_ID,end_station_ID
0,3000,Virtual Station,4/23/2015,Active,3000,Virtual Station,3000,Virtual Station,3000,3000
1,3004,Municipal Services Building Plaza,4/23/2015,Active,3004,Municipal Services Building Plaza,3004,Municipal Services Building Plaza,3004,3004
2,3005,"Welcome Park, NPS",4/23/2015,Active,3005,"Welcome Park, NPS",3005,"Welcome Park, NPS",3005,3005
3,3006,40th & Spruce,4/23/2015,Active,3006,40th & Spruce,3006,40th & Spruce,3006,3006
4,3007,"11th & Pine, Kahn Park",4/23/2015,Active,3007,"11th & Pine, Kahn Park",3007,"11th & Pine, Kahn Park",3007,3007


In [32]:
#####Zuordnungstabelle start_station mit Station_Name
startStationName=dsStationsInfo[['start_station', 'start_station_name']]
startStationName.head()

,start_station,start_station_name
0,3000,Virtual Station
1,3004,Municipal Services Building Plaza
2,3005,"Welcome Park, NPS"
3,3006,40th & Spruce
4,3007,"11th & Pine, Kahn Park"


In [25]:
#####Zuordnungstabelle end_station mit Station_Name
endStationName=dsStationsInfo[['end_station', 'end_station_name']]
endStationName.head()

,end_station,end_station_name
0,3000,Virtual Station
1,3004,Municipal Services Building Plaza
2,3005,"Welcome Park, NPS"
3,3006,40th & Spruce
4,3007,"11th & Pine, Kahn Park"


In [27]:
###MERGE###
# using merge function by setting how='inner'
output1 = pd.merge(dsStation1, startStationName, 
                   on='start_station', 
                   how='inner')
  
# displaying result
output1.tail()
#print(output1)

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,start_station_name
98980,229675538,16,2018-02-05 12:20:00,2018-02-05 12:36:00,3014,39.958858,-75.173691,3008,39.980782,-75.150551,11735,30,One Way,Indego30,"21st & Winter, Franklin Institute"
98981,230948270,1,2018-02-11 16:12:00,2018-02-11 16:13:00,3014,39.958858,-75.173691,3014,39.958858,-75.173691,3305,30,Round Trip,Indego30,"21st & Winter, Franklin Institute"
98982,233071565,7,2018-02-21 14:32:00,2018-02-21 14:39:00,3014,39.958858,-75.173691,3041,39.968491,-75.135460,5159,30,One Way,Indego30,"21st & Winter, Franklin Institute"
98983,236515417,1,2018-03-08 15:30:00,2018-03-08 15:31:00,3014,39.958858,-75.173691,3014,39.958858,-75.173691,5272,30,Round Trip,Indego30,"21st & Winter, Franklin Institute"
98984,236515410,1,2018-03-08 15:35:00,2018-03-08 15:36:00,3014,39.958858,-75.173691,3014,39.958858,-75.173691,5272,30,Round Trip,Indego30,"21st & Winter, Franklin Institute"


In [28]:
dfYear = pd.concat([dsStation1, dsStation2, dsStation3, dsStation4])
dfYear


,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type
0,223869188,18,2018-01-01 00:24:00,2018-01-01 00:42:00,3124,39.952950,-75.139793,3073,39.961430,-75.152420,3708,30,One Way,Indego30,NaN
1,223905597,572,2018-01-01 00:38:00,2018-01-01 10:10:00,3023,39.950481,-75.172859,3066,39.945610,-75.173477,3288,365,One Way,Indego365,NaN
2,223872811,22,2018-01-01 00:48:00,2018-01-01 01:10:00,3026,39.941380,-75.145638,3023,39.950481,-75.172859,11735,30,One Way,Indego30,NaN
3,223872810,21,2018-01-01 01:03:00,2018-01-01 01:24:00,3045,39.947922,-75.162369,3037,39.954239,-75.161377,5202,30,One Way,Indego30,NaN
4,223872809,4,2018-01-01 01:05:00,2018-01-01 01:09:00,3115,39.972630,-75.167572,3058,39.967159,-75.170013,5142,30,One Way,Indego30,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142867,306773867,46,2018-12-31 23:18:00,2019-01-01 00:04:00,3037,39.954239,-75.161377,3037,39.954239,-75.161377,14519,1,Round Trip,Day Pass,standard
142868,306773866,41,2018-12-31 23:25:00,2019-01-01 00:06:00,3168,39.951340,-75.173943,3168,39.951340,-75.173943,11891,30,Round Trip,Indego30,standard
142869,306787530,438,2018-12-31 23:26:00,2019-01-01 06:44:00,3026,39.941380,-75.145638,3026,39.941380,-75.145638,05410,30,Round Trip,Indego30,standard
142870,306767956,14,2018-12-31 23:33:00,2018-12-31 23:47:00,3063,39.946331,-75.169800,3150,39.946331,-75.169800,02701,1,One Way,Day Pass,standard


In [29]:
dfYear.to_csv("Year2018.csv")

In [30]:
fourquarters = pd.read_csv('Year2018.csv', low_memory=False)
fourquarters["Coordinates_Start"] = list(zip(fourquarters["start_lat"].round(4),fourquarters["start_lon"].round(4)))
fourquarters["Coordinates_End"] = list(zip(fourquarters["end_lat"].round(4),fourquarters["end_lon"].round(4)))


In [34]:
###MERGE###
#Create Start Name
# using merge function by setting how='inner'
fourquarters3 = pd.merge(fourquarters, startStationName, 
                   on='start_station', 
                   how='left')
  
# displaying result
fourquarters3.tail()
#print(output1)

,Unnamed: 0,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Coordinates_Start,Coordinates_End,start_station_name
671813,142867,306773867,46,2018-12-31 23:18:00,2019-01-01 00:04:00,3037,39.954239,-75.161377,3037,39.954239,-75.161377,14519,1,Round Trip,Day Pass,standard,"(39.9542, -75.1614)","(39.9542, -75.1614)",Pennsylvania Convention Center
671814,142868,306773866,41,2018-12-31 23:25:00,2019-01-01 00:06:00,3168,39.951340,-75.173943,3168,39.951340,-75.173943,11891,30,Round Trip,Indego30,standard,"(39.9513, -75.1739)","(39.9513, -75.1739)",20th & Sansom
671815,142869,306787530,438,2018-12-31 23:26:00,2019-01-01 06:44:00,3026,39.941380,-75.145638,3026,39.941380,-75.145638,05410,30,Round Trip,Indego30,standard,"(39.9414, -75.1456)","(39.9414, -75.1456)",2nd & South
671816,142870,306767956,14,2018-12-31 23:33:00,2018-12-31 23:47:00,3063,39.946331,-75.169800,3150,39.946331,-75.169800,02701,1,One Way,Day Pass,standard,"(39.9463, -75.1698)","(39.9463, -75.1698)",17th & Pine
671817,142871,306767955,10,2018-12-31 23:39:00,2018-12-31 23:49:00,3007,39.945171,-75.159927,3049,39.945171,-75.159927,03471,30,One Way,Indego30,standard,"(39.9452, -75.1599)","(39.9452, -75.1599)","11th & Pine, Kahn Park"


In [36]:
###MERGE###
#Create Start Name
# using merge function by setting how='inner'
fourquarters4 = pd.merge(fourquarters3, endStationName, 
                   on='end_station', 
                   how='left')
  
# displaying result
fourquarters4.tail()
#print(output1)

,Unnamed: 0,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Coordinates_Start,Coordinates_End,start_station_name,end_station_name
671813,142867,306773867,46,2018-12-31 23:18:00,2019-01-01 00:04:00,3037,39.954239,-75.161377,3037,39.954239,-75.161377,14519,1,Round Trip,Day Pass,standard,"(39.9542, -75.1614)","(39.9542, -75.1614)",Pennsylvania Convention Center,Pennsylvania Convention Center
671814,142868,306773866,41,2018-12-31 23:25:00,2019-01-01 00:06:00,3168,39.951340,-75.173943,3168,39.951340,-75.173943,11891,30,Round Trip,Indego30,standard,"(39.9513, -75.1739)","(39.9513, -75.1739)",20th & Sansom,20th & Sansom
671815,142869,306787530,438,2018-12-31 23:26:00,2019-01-01 06:44:00,3026,39.941380,-75.145638,3026,39.941380,-75.145638,05410,30,Round Trip,Indego30,standard,"(39.9414, -75.1456)","(39.9414, -75.1456)",2nd & South,2nd & South
671816,142870,306767956,14,2018-12-31 23:33:00,2018-12-31 23:47:00,3063,39.946331,-75.169800,3150,39.946331,-75.169800,02701,1,One Way,Day Pass,standard,"(39.9463, -75.1698)","(39.9463, -75.1698)",17th & Pine,"15th & Castle, DiSilvestro Playground"
671817,142871,306767955,10,2018-12-31 23:39:00,2018-12-31 23:49:00,3007,39.945171,-75.159927,3049,39.945171,-75.159927,03471,30,One Way,Indego30,standard,"(39.9452, -75.1599)","(39.9452, -75.1599)","11th & Pine, Kahn Park",Foglietta Plaza


In [15]:
fourquarters.tail()

,Unnamed: 0,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Coordinates_Start,Coordinates_End
671813,142867,306773867,46,2018-12-31 23:18:00,2019-01-01 00:04:00,3037,39.954239,-75.161377,3037,39.954239,-75.161377,14519,1,Round Trip,Day Pass,standard,"(39.9542, -75.1614)","(39.9542, -75.1614)"
671814,142868,306773866,41,2018-12-31 23:25:00,2019-01-01 00:06:00,3168,39.951340,-75.173943,3168,39.951340,-75.173943,11891,30,Round Trip,Indego30,standard,"(39.9513, -75.1739)","(39.9513, -75.1739)"
671815,142869,306787530,438,2018-12-31 23:26:00,2019-01-01 06:44:00,3026,39.941380,-75.145638,3026,39.941380,-75.145638,05410,30,Round Trip,Indego30,standard,"(39.9414, -75.1456)","(39.9414, -75.1456)"
671816,142870,306767956,14,2018-12-31 23:33:00,2018-12-31 23:47:00,3063,39.946331,-75.169800,3150,39.946331,-75.169800,02701,1,One Way,Day Pass,standard,"(39.9463, -75.1698)","(39.9463, -75.1698)"
671817,142871,306767955,10,2018-12-31 23:39:00,2018-12-31 23:49:00,3007,39.945171,-75.159927,3049,39.945171,-75.159927,03471,30,One Way,Indego30,standard,"(39.9452, -75.1599)","(39.9452, -75.1599)"


In [37]:
# For performance reasons we select a single car only - Pick randomly from the list
bike_list = fourquarters4["bike_id"].unique()

bike_list



array(['3708', '3288', '11735', ..., '14635', '02671', '14620'],
      dtype=object)

In [38]:
selected_bike_ID = random.choice(bike_list)
print(selected_bike_ID)

3436


In [39]:
#define new map

PA_heat_map = folium.Map(location=(39.953, -75.1398),  tiles='Stamen Terrain', 
                       zoom_start=11, control_scale=True, max_zoom=20)

# add heat map

PA_heat_map.add_child(plugins.HeatMap(fourquarters4[fourquarters4["bike_id"]==selected_bike_ID]["Coordinates_Start"], radius=10))

PA_heat_map

In [40]:
#define new map

PA_heat_map = folium.Map(location=(39.953, -75.1398),  tiles='Stamen Terrain', 
                       zoom_start=11, control_scale=True, max_zoom=20)

# add heat map

PA_heat_map.add_child(plugins.HeatMap(fourquarters4[fourquarters4["bike_id"]==selected_bike_ID]["Coordinates_Start"], radius=10))

PA_heat_map

In [41]:
PA_Nov_bike_select = fourquarters4[fourquarters4["bike_id"]==selected_bike_ID]

PA_map_trajectory = folium.Map(location=(39.953, -75.1398),  tiles='Stamen Terrain', 
                       zoom_start=9, control_scale=True, max_zoom=20)

# draw trajectory for a selected bike

folium.PolyLine(PA_Nov_bike_select["Coordinates_Start"],color="blue", weight=5, opacity=0.8, ).add_to(PA_map_trajectory)
PA_map_trajectory

In [21]:
fourquarters_clean = fourquarters4[pd.isnull(fourquarters.Coordinates_Start) == False]
fourquarters_clean.tail()

,Unnamed: 0,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Coordinates_Start,Coordinates_End
671813,142867,306773867,46,2018-12-31 23:18:00,2019-01-01 00:04:00,3037,39.954239,-75.161377,3037,39.954239,-75.161377,14519,1,Round Trip,Day Pass,standard,"(39.9542, -75.1614)","(39.9542, -75.1614)"
671814,142868,306773866,41,2018-12-31 23:25:00,2019-01-01 00:06:00,3168,39.951340,-75.173943,3168,39.951340,-75.173943,11891,30,Round Trip,Indego30,standard,"(39.9513, -75.1739)","(39.9513, -75.1739)"
671815,142869,306787530,438,2018-12-31 23:26:00,2019-01-01 06:44:00,3026,39.941380,-75.145638,3026,39.941380,-75.145638,05410,30,Round Trip,Indego30,standard,"(39.9414, -75.1456)","(39.9414, -75.1456)"
671816,142870,306767956,14,2018-12-31 23:33:00,2018-12-31 23:47:00,3063,39.946331,-75.169800,3150,39.946331,-75.169800,02701,1,One Way,Day Pass,standard,"(39.9463, -75.1698)","(39.9463, -75.1698)"
671817,142871,306767955,10,2018-12-31 23:39:00,2018-12-31 23:49:00,3007,39.945171,-75.159927,3049,39.945171,-75.159927,03471,30,One Way,Indego30,standard,"(39.9452, -75.1599)","(39.9452, -75.1599)"


In [47]:
fourquarters4.dtypes
#fourquarters_clean = fourquarters4[pd.to_numeric(duration, downcast='integer') >= 0]
#fourquarters_clean.tail()

Unnamed: 0               int64
trip_id                  int64
duration                 int64
start_time              object
end_time                object
start_station            int64
start_lat              float64
start_lon              float64
end_station              int64
end_lat                float64
end_lon                float64
bike_id                 object
plan_duration            int64
trip_route_category     object
passholder_type         object
bike_type               object
Coordinates_Start       object
Coordinates_End         object
start_station_name      object
end_station_name        object
dtype: object

In [60]:
filt = (fourquarters4['duration']>0)
        #fourquarters_clean.tail()

In [62]:
filt.tail()

671813    True
671814    True
671815    True
671816    True
671817    True
Name: duration, dtype: bool

In [64]:
fourquarters4['start_time'] = pd.to_datetime(fourquarters4['start_time'])
fourquarters4['end_time'] = pd.to_datetime(fourquarters4['end_time'])
fourquarters4['duration_of_rent']= fourquarters4['end_time']-fourquarters4['start_time']
fourquarters4['duration_of_rent'].min()

Timedelta('-1 days +23:06:00')

In [70]:
#fourquarters4[fourquarters4["duration_of_rent"] < '0 days 00:00:00']
filt2 = (fourquarters4['duration_of_rent']<'0 days 00:00:00')

In [71]:
fourquarters4[filt2]

,Unnamed: 0,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,...,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,Coordinates_Start,Coordinates_End,start_station_name,end_station_name,duration_of_rent
603513,74567,302630902,59,2018-11-04 01:21:00,2018-11-04 01:20:00,3011,39.959492,-75.196899,3009,39.959492,...,05262,30,One Way,Indego30,standard,"(39.9595, -75.1969)","(39.9595, -75.1969)",38th & Powelton,33rd & Market,-1 days +23:59:00
603524,74578,302630894,32,2018-11-04 01:42:00,2018-11-04 01:14:00,3190,39.948921,-75.169907,3108,39.948921,...,03718,30,One Way,Indego30,standard,"(39.9489, -75.1699)","(39.9489, -75.1699)",17th & Locust,15th & Market,-1 days +23:32:00
603527,74581,302630892,13,2018-11-04 01:47:00,2018-11-04 01:00:00,3167,39.950249,-75.160278,3015,39.950249,...,14540,1,One Way,Day Pass,standard,"(39.9502, -75.1603)","(39.9502, -75.1603)",12th & Chestnut,"4th & Walnut, NPS",-1 days +23:13:00
603529,74583,302630891,12,2018-11-04 01:48:00,2018-11-04 01:00:00,3010,39.947109,-75.166183,3099,39.947109,...,03565,30,One Way,Indego30,standard,"(39.9471, -75.1662)","(39.9471, -75.1662)",15th & Spruce,4th & Washington,-1 days +23:12:00
603530,74584,302630889,11,2018-11-04 01:49:00,2018-11-04 01:00:00,3167,39.950249,-75.160278,3015,39.950249,...,11895,1,One Way,Day Pass,standard,"(39.9502, -75.1603)","(39.9502, -75.1603)",12th & Chestnut,"4th & Walnut, NPS",-1 days +23:11:00
603532,74586,302630887,9,2018-11-04 01:51:00,2018-11-04 01:00:00,3167,39.950249,-75.160278,3015,39.950249,...,14487,1,One Way,Day Pass,standard,"(39.9502, -75.1603)","(39.9502, -75.1603)",12th & Chestnut,"4th & Walnut, NPS",-1 days +23:09:00
603536,74590,302630886,6,2018-11-04 01:54:00,2018-11-04 01:00:00,3086,39.940189,-75.166908,3052,39.940189,...,03448,365,One Way,Indego365,standard,"(39.9402, -75.1669)","(39.9402, -75.1669)",Broad & Christian,9th & Locust,-1 days +23:06:00
